<a href="https://colab.research.google.com/github/Senbonsakurakageyoshi1/AgrifieldNet/blob/main/Starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# A Baseline Model for the AgrifieldNet India Competition

This notebook walks you through the steps to load the data and build a baseline model using Random Forests for `AgrifieldNet India Competition`.

## Radiant MLHub API


The Radiant MLHub API gives access to open Earth imagery training data for machine learning applications. You can learn more about the repository at the [Radiant MLHub site](https://mlhub.earth) and about the organization behind it at the [Radiant Earth Foundation site](https://radiant.earth).

Full documentation for the API is available at [docs.mlhub.earth](docs.mlhub.earth).

Each item in our collection is explained in json format compliant with [STAC](https://stacspec.org/) [label extension](https://github.com/radiantearth/stac-spec/tree/master/extensions/label) definition.

## Dependencies

This notebook utilizes the [`radiant-mlhub` Python client](https://pypi.org/project/radiant-mlhub/) for interacting with the API. This notebook also utilizes the [`pandas` library](https://pandas.pydata.org/). If you are running this notebooks using Binder, then these dependencies have already been installed. If you are running this notebook locally, you will need to install these yourself.

See the official [`radiant-mlhub` docs](https://radiant-mlhub.readthedocs.io/) for more documentation of the full functionality of that library.

In [3]:
%%writefile requirements.txt

pandas==1.3.5
scikit-learn==1.0.2
tqdm==4.64.0
rasterio==1.2.0
numpy==1.21.6
radiant_mlhub==0.4.1

Writing requirements.txt


In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 19.1 MB 1.1 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 146 kB 74.0 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [19]:
import os
import json
import getpass
import rasterio
import numpy as np
import pandas as pd
from tqdm import tqdm
from radiant_mlhub import Dataset
import tarfile
import shutil
import numpy as np
import pandas as pd

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
data_dir = 'data'

In [22]:
collection_name = 'ref_agrifieldnet_competition_v1'

source_collection = f'{collection_name}_source'
train_label_collection = f'{collection_name}_labels_train'
test_label_collection = f'{collection_name}_labels_test'

In [9]:
Key = "92926e263b5fa39b11501fa98ce04086d628a9ec40cc28bbc4ceddbf2a823847"

if not os.path.exists(data_dir):
  os.environ['MLHUB_API_KEY'] =  getpass.getpass(prompt="MLHub API Key: ")

  dataset = Dataset.fetch(collection_name)
  dataset.download(output_dir=data_dir)
  for fn in os.listdir(data_dir):
    with tarfile.open(os.path.join(data_dir, fn)) as f:
      f.extractall(data_dir + '/') 
    os.remove(os.path.join(data_dir, fn))
else:
  print("Dataset already exists")

MLHub API Key: ··········


  0%|          | 0/0.5 [00:00<?, ?M/s]

  0%|          | 0/324.3 [00:00<?, ?M/s]

  0%|          | 0/1.3 [00:00<?, ?M/s]

In [10]:
#Create Dataframe

In [11]:
train_paths = os.listdir(os.path.join(data_dir, train_label_collection))
train_ids = [fn.split('_')[-1] for fn in train_paths if 'labels_train' in fn]

field_paths = [f'{data_dir}/{train_label_collection}/{train_label_collection}_{i}/field_ids.tif' 
               for i in train_ids]
label_paths = [f'{data_dir}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' 
               for i in train_ids]
source_paths = [f'{data_dir}/{source_collection}/{source_collection}_{i}/' 
               for i in train_ids]

In [12]:
train_data = pd.DataFrame(np.array([train_ids, field_paths, label_paths, source_paths]).transpose(), 
                          columns=['folder_id', 'field_path', 'label_path', 'source_path'])
train_data.head()

,folder_id,field_path,label_path,source_path
0,738c6,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
1,b938c,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
2,312bb,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
3,167fc,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...
4,c94f1,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_labels_tr...,data/ref_agrifieldnet_competition_v1_source/re...


Extract Field crop

In [13]:
def extract_field_crop_data(data):
  field_ids = []
  crop_type = []

  for i in tqdm(range(len(data))):
      print(i)
      with rasterio.open(data['field_path'].iloc[i]) as src:
          field_data = src.read()[0]
      with rasterio.open(data['label_path'].iloc[i]) as src:
          crop_data = src.read()[0]

      for field_id in np.unique(field_data)[1:]:
          ind = np.where(field_data == field_id)
          field_ids.append(field_id)
          crop_type.append(np.unique(crop_data[ind])[-1])

  df = pd.DataFrame(np.array([field_ids, crop_type]).transpose(),
                    columns=['field_id', 'crop_type'])
  return df

In [14]:
df = extract_field_crop_data(train_data)
df.head()

  3%|▎         | 33/1165 [00:00<00:06, 164.87it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37


  6%|▌         | 72/1165 [00:00<00:05, 182.70it/s]

38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75


 10%|▉         | 111/1165 [00:00<00:05, 183.06it/s]

...

 96%|█████████▌| 1113/1165 [00:06<00:00, 186.05it/s]

1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114
1115
1116
1117
1118
1119
1120
1121
1122
1123
1124
1125
1126
1127
1128
1129
1130
1131


100%|██████████| 1165/1165 [00:06<00:00, 182.37it/s]

1132
1133
1134
1135
1136
1137
1138
1139
1140
1141
1142
1143
1144
1145
1146
1147
1148
1149
1150
1151
1152
1153
1154
1155
1156
1157
1158
1159
1160
1161
1162
1163
1164


,field_id,crop_type
0,6714,4
1,6715,1
2,7002,1
3,5942,2
4,1990,1


In [15]:
!python --version

Python 3.7.14


# DOWNLOAD DATA FROM MLHUB

In [17]:
#For simplicity we select 4 out 12 bands for the this baseline model

Full_bands = ['B01', 'B02', 'B03', 'B04','B05', 'B06', 'B07', 'B08','B8A', 'B09', 'B11', 'B12']

selected_bands = Full_bands#[1:4]  + [Full_bands[-5]]  #'B02', 'B03', 'B04', 'B08'
selected_bands

['B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B8A',
 'B09',
 'B11',
 'B12']

## Prepare Train data

- Load collection.json in labels_train collection's path and retrieve all unique folder ids into a list.
- Use unique folder ids to create a list of field.tif and raster_labels.tif paths for all tiles.
- Create competition_train_data dataframe for folder_ids and field_paths
- Create field_crop_pair dataframe using field_crop_extractor.
- Create train_data dataframe using the feature_extractor with argsss (competition_train_data, source_collection)
- Group processed dataset by fields and find the pixel average of across the entire field 
- Merge train_data dataframe and field_crop_pair dataframe on field_id
- Split train_df dataframe for model training and evaluation 

In [23]:
# #load collection json and retrieve all unique folder ids 
# #use all unique folder ids to create a list of field and label paths for all tiles

# with open (f'{main}/{train_label_collection}/collection.json') as f:
#     train_json = json.load(f)
    
# train_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in train_json['links'][4:]]

# train_field_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/field_ids.tif' for i in train_folder_ids]
# train_label_paths = [f'{main}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' for i in train_folder_ids]

In [24]:
# #create dataset for folder_ids and field_paths

# competition_train_data = pd.DataFrame(train_folder_ids, columns=['unique_folder_id'])
# competition_train_data['field_paths'] = train_field_paths
# competition_train_data.head()

In [25]:
# PREPROCESS FIELDS AND CROPS IN TILES FOR TRAININIG

In [26]:
#Extract field_crop Pairs 

def field_crop_extractor(crop_field_files):
    field_crops = {}

    for label_field_file in tqdm(crop_field_files):
        with rasterio.open(f'{data_dir}/{train_label_collection}/{train_label_collection}_{label_field_file}/field_ids.tif') as src:
            field_data = src.read()[0]
        with rasterio.open(f'{data_dir}/{train_label_collection}/{train_label_collection}_{label_field_file}/raster_labels.tif') as src:
            crop_data = src.read()[0]
    
        for x in range(0, crop_data.shape[0]):
            for y in range(0, crop_data.shape[1]):
                field_id = str(field_data[x][y])
                field_crop = crop_data[x][y]

                if field_crops.get(field_id) is None:
                    field_crops[field_id] = []

                if field_crop not in field_crops[field_id]:
                    field_crops[field_id].append(field_crop)
    
    field_crop_map  =[[k, v[0]]  for k, v in field_crops.items() ]
    field_crop = pd.DataFrame(field_crop_map , columns=['field_id','crop_id'])

    return field_crop[field_crop['field_id']!='0']

In [27]:
field_crop_pair = field_crop_extractor(train_ids)
field_crop_pair.head()

100%|██████████| 1165/1165 [01:25<00:00, 13.56it/s]


,field_id,crop_id
1,6714,4
2,6715,1
3,7002,1
4,5942,2
5,1990,1


In [28]:
field_crop_pair.shape

(5551, 2)

In [29]:
# Our goal is developing a pixel-based Random Forest model. So we will create an X variable
# such that, each row is a pixel and each column is one of the band observations mapped to its corresponding field. 


img_sh = 256
n_selected_bands= len(selected_bands)

n_obs = 1  #imagery per chip(no time series)

def feature_extractor(data_ ,   path ):
    '''
        data_: Dataframe with 'field_paths' and 'unique_folder_id' columns
        path: Path to source collections files

        returns: pixel dataframe with corresponding field_ids
        '''
    
    X = np.empty((0, n_selected_bands * n_obs))
    X_tile = np.empty((img_sh * img_sh, 0))
    X_arrays = []
        
    field_ids = np.empty((0, 1))

    for idx, tile_id in tqdm(enumerate(data_['folder_id'])):
        
        field_src =   rasterio.open( data_['field_path'].values[idx])
        field_array = field_src.read(1)
        field_ids = np.append(field_ids, field_array.flatten())
        
        
        bands_src = [rasterio.open(f'{data_dir}/{path}/{path}_{tile_id}/{band}.tif') for band in selected_bands]
        bands_array = [np.expand_dims(band.read(1).flatten(), axis=1) for band in bands_src]
        
        X_tile = np.hstack(bands_array)

        X_arrays.append(X_tile)
        

    X = np.concatenate(X_arrays)
    
    data = pd.DataFrame(X, columns=selected_bands)

    data['field_id'] = field_ids

    return data[data['field_id']!=0]

In [30]:
train_data = feature_extractor(train_data, source_collection)
train_data.head()

1165it [01:00, 19.17it/s]


,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,field_id
53962,45,39,35,30,33,48,58,57,63,11,49,31,6714.0
53963,45,39,35,31,33,48,58,55,63,11,49,31,6714.0
53964,46,40,36,32,34,48,57,55,62,10,51,34,6714.0
54218,45,39,35,32,33,48,58,53,63,11,49,31,6714.0
54219,45,40,35,33,33,48,58,52,63,11,49,31,6714.0


In [31]:
# Each field has several pixels in| the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id

train_data_grouped = train_data.groupby(['field_id']).mean().reset_index()
train_data_grouped.field_id = [str(int(i)) for i in train_data_grouped.field_id.values]
train_data_grouped.head()

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12
0,1,45.000000,42.444444,42.722222,48.000000,49.666667,58.000000,65.222222,60.277778,71.944444,12.000000,80.277778,61.333333
1,2,45.000000,42.000000,42.166667,47.666667,49.250000,59.916667,69.000000,63.916667,76.333333,12.833333,79.916667,56.750000
2,3,45.000000,42.687500,43.500000,49.187500,51.437500,62.875000,71.625000,66.625000,79.312500,13.000000,82.125000,58.062500
3,4,45.866667,42.466667,43.800000,47.733333,49.466667,59.733333,68.133333,62.600000,73.466667,11.266667,77.600000,55.000000
4,5,46.000000,43.238095,45.238095,49.285714,50.904762,60.904762,68.952381,63.380952,74.547619,11.333333,77.452381,55.809524


In [32]:
# merge pixel dataframe to field_crop_pair dataframe

train_df = pd.merge(train_data_grouped, field_crop_pair , on='field_id' )
train_df.head()

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,crop_id
0,1,45.000000,42.444444,42.722222,48.000000,49.666667,58.000000,65.222222,60.277778,71.944444,12.000000,80.277778,61.333333,1
1,2,45.000000,42.000000,42.166667,47.666667,49.250000,59.916667,69.000000,63.916667,76.333333,12.833333,79.916667,56.750000,1
2,3,45.000000,42.687500,43.500000,49.187500,51.437500,62.875000,71.625000,66.625000,79.312500,13.000000,82.125000,58.062500,1
3,4,45.866667,42.466667,43.800000,47.733333,49.466667,59.733333,68.133333,62.600000,73.466667,11.266667,77.600000,55.000000,2
4,5,46.000000,43.238095,45.238095,49.285714,50.904762,60.904762,68.952381,63.380952,74.547619,11.333333,77.452381,55.809524,2


In [33]:
train_df.tail(8)

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,crop_id
5543,7322,45.000000,36.500000,32.250000,26.750000,31.000000,54.000000,67.875000,64.500000,75.000000,13.000000,49.625000,26.750000,4
5544,7323,45.000000,36.000000,31.000000,24.090909,27.818182,56.363636,75.454545,71.090909,82.181818,13.000000,41.909091,20.909091,4
5545,7324,45.000000,36.533333,31.666667,25.333333,29.133333,55.333333,70.800000,67.400000,77.600000,13.866667,43.066667,21.466667,4
5546,7326,46.384615,39.000000,34.576923,30.653846,33.153846,51.076923,62.538462,59.269231,66.846154,12.461538,50.615385,33.923077,9
5547,7327,46.000000,37.851852,32.629630,26.555556,29.296296,51.185185,65.518519,63.925926,70.444444,12.777778,40.740741,21.518519,9
5548,7328,47.000000,40.100000,35.100000,29.650000,31.950000,51.600000,65.050000,60.550000,70.700000,11.000000,39.550000,20.000000,9
5549,7331,46.652174,40.130435,35.130435,30.000000,32.608696,52.347826,66.347826,62.608696,71.521739,11.000000,44.434783,25.217391,9
5550,7332,46.076923,39.653846,35.230769,29.423077,31.846154,49.615385,58.615385,53.346154,60.923077,10.461538,35.538462,18.615385,36


In [34]:
train_df.shape

(5551, 14)

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [36]:
# split data for model training and evaluation 

X_train, X_test, y_train, y_test =  train_test_split(train_df.drop(['field_id', 'crop_id'], axis=1), train_df['crop_id'] , test_size=0.25, random_state=42)

In [37]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4163, 12), (1388, 12), (4163,), (1388,))

# MODEL TRAINING

In [38]:
# We ran a simple hyperparameter tuning for the number of trees, and concluded to use:
n_trees = 300

In [104]:
# Fitting the RF model
rf = RandomForestClassifier(n_estimators = n_trees,   oob_score = True, n_jobs = -1,random_state =50,max_features = "auto", min_samples_leaf = 10)


In [105]:
rf.fit(X_train, y_train.astype(int))

RandomForestClassifier(min_samples_leaf=3, n_estimators=300, n_jobs=-1,
                       oob_score=True, random_state=50)

In [107]:
# trained classes

rf.classes_

array([ 1,  2,  3,  4,  5,  6,  8,  9, 13, 14, 15, 16, 36])

Support Vector Machine

In [41]:
from sklearn.svm import SVC

In [42]:

svm_model_linear = SVC(kernel = 'linear', C = 1)
svm_model_linear.fit(X_train, y_train.astype(int))

SVC(C=1, kernel='linear')

K nearest neighbor

In [43]:
# training a KNN classifier
from sklearn.neighbors import KNeighborsClassifier

In [44]:
knn = KNeighborsClassifier(n_neighbors = 7)
knn.fit(X_train, y_train.astype(int))

KNeighborsClassifier(n_neighbors=7)

Naive bayes

In [45]:
# training a Naive Bayes classifier
from sklearn.naive_bayes import GaussianNB

In [46]:

gnb = GaussianNB()
gnb.fit(X_train, y_train.astype(int))

GaussianNB()

OneVsRest Algorithm

In [47]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

In [48]:
# Onerest = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))

In [49]:
Onerest3 = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=3))

In [50]:
Onerest2 = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=2))

In [51]:
# You may need to use MultiLabelBinarizer to encode your variables from arrays [[x, y, z]] to a multilabel 
# format before training.
mlb = MultiLabelBinarizer()
#

Onerest2.fit(X_train, y_train.astype(int))

OneVsRestClassifier(estimator=XGBClassifier(max_depth=2, n_jobs=-1))

# MODEL EVALUATION

In [108]:
from sklearn.metrics import classification_report
y_pred_crop1 = rf.predict(X_test)

print(classification_report(y_test,y_pred_crop1))

              precision    recall  f1-score   support

           1       0.65      0.84      0.73       516
           2       0.43      0.39      0.41       235
           3       0.00      0.00      0.00        27
           4       0.79      0.74      0.77       422
           5       0.00      0.00      0.00         4
           6       0.29      0.11      0.16        44
           8       0.00      0.00      0.00        15
           9       0.70      0.87      0.78        78
          13       0.00      0.00      0.00        15
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00         3
          36       1.00      0.05      0.10        20

    accuracy                           0.66      1388
   macro avg       0.30      0.23      0.23      1388
weighted avg       0.62      0.66      0.62      1388



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [109]:
print(abs(y_pred_crop1-y_test))

1919    0
5216    0
4251    1
230     1
3360    0
       ..
4704    0
1669    0
4058    0
2972    0
4509    1
Name: crop_id, Length: 1388, dtype: int64


In [53]:
from sklearn.metrics import classification_report
y_pred_crop = svm_model_linear.predict(X_test)

print(classification_report(y_test,y_pred_crop))

              precision    recall  f1-score   support

           1       0.57      0.92      0.71       516
           2       0.41      0.05      0.09       235
           3       0.00      0.00      0.00        27
           4       0.74      0.75      0.74       422
           5       0.00      0.00      0.00         4
           6       0.00      0.00      0.00        44
           8       0.00      0.00      0.00        15
           9       0.68      0.86      0.76        78
          13       0.00      0.00      0.00        15
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00         3
          36       0.00      0.00      0.00        20

    accuracy                           0.63      1388
   macro avg       0.18      0.20      0.18      1388
weighted avg       0.55      0.63      0.55      1388



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
from sklearn.metrics import classification_report
y_pred_crop = knn.predict(X_test)

print(classification_report(y_test,y_pred_crop))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1       0.61      0.81      0.70       516
           2       0.40      0.34      0.37       235
           3       0.00      0.00      0.00        27
           4       0.81      0.72      0.76       422
           5       0.00      0.00      0.00         4
           6       0.18      0.07      0.10        44
           8       0.00      0.00      0.00        15
           9       0.65      0.85      0.74        78
          13       0.25      0.13      0.17        15
          14       0.00      0.00      0.00         2
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00         3
          36       0.25      0.05      0.08        20

    accuracy                           0.63      1388
   macro avg       0.24      0.23      0.22      1388
weighted avg       0.59      0.63      0.60      1388



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [55]:
from sklearn.metrics import classification_report
y_pred_crop = gnb.predict(X_test)

print(classification_report(y_test,y_pred_crop))

              precision    recall  f1-score   support

           1       0.57      0.63      0.60       516
           2       0.08      0.00      0.01       235
           3       0.08      0.26      0.13        27
           4       0.86      0.48      0.62       422
           5       0.00      0.00      0.00         4
           6       0.12      0.55      0.20        44
           8       0.10      0.47      0.17        15
           9       0.48      0.97      0.64        78
          13       0.25      0.07      0.11        15
          14       0.00      0.00      0.00         2
          15       0.04      0.14      0.06         7
          16       0.00      0.00      0.00         3
          36       0.05      0.10      0.07        20

    accuracy                           0.47      1388
   macro avg       0.20      0.28      0.20      1388
weighted avg       0.53      0.47      0.46      1388



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
from sklearn.metrics import classification_report
y_pred_crop = Onerest3.predict(X_test)

print(classification_report(y_test,y_pred_crop))

NotFittedError: ignored

## Prepare Test data

- Load collection json and retrieve all unique folder ids 
- Use unique folder ids to create a list of field.tif paths for all tiles
- Create competition_test_data dataframe for folder_ids and field_paths
- Create test_data dataframe using the feature_extractor with argsss (competition_test_data, source_collection)
- Group processed dataset by fields and find the pixel average of across the entire field 

In [57]:
data_dir

'data'

In [58]:
# with open (f'{data_dir}/{test_label_collection}/collection.json') as f:
#     test_json = json.load(f)
    
# test_folder_ids = [i['href'].split('_')[-1].split('.')[0] for i in test_json['links'][4:]]

# test_field_paths = [f'{data_dir}/{test_label_collection}/{test_label_collection}_{i}/field_ids.tif' for i in test_folder_ids]

In [59]:
# competition_test_data = pd.DataFrame(test_folder_ids , columns=['folder_id'])
# competition_test_data['field_path'] = test_field_paths
# competition_test_data.head()

In [60]:
test_paths = os.listdir(os.path.join(data_dir, test_label_collection))
test_folder_ids = [fn.split('_')[-1] for fn in test_paths if 'labels_test' in fn]

test_field_paths = [f'{data_dir}/{test_label_collection}/{test_label_collection}_{i}/field_ids.tif' 
               for i in test_folder_ids]
# label_paths = [f'{data_dir}/{train_label_collection}/{train_label_collection}_{i}/raster_labels.tif' 
#                for i in train_ids]
# source_paths = [f'{data_dir}/{source_collection}/{source_collection}_{i}/' 
#                for i in train_ids]

In [61]:
competition_test_data = pd.DataFrame(test_folder_ids , columns=['folder_id'])
competition_test_data['field_path'] = test_field_paths
competition_test_data.head()

,folder_id,field_path
0,d754d,data/ref_agrifieldnet_competition_v1_labels_te...
1,66c64,data/ref_agrifieldnet_competition_v1_labels_te...
2,41c19,data/ref_agrifieldnet_competition_v1_labels_te...
3,5c339,data/ref_agrifieldnet_competition_v1_labels_te...
4,87f98,data/ref_agrifieldnet_competition_v1_labels_te...


In [62]:
test_data = feature_extractor(competition_test_data,  source_collection)
test_data.head()

707it [00:29, 23.70it/s]


,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12,field_id
4109,39,32,30,26,32,49,56,53,64,12,58,33,4011.0
4364,39,32,30,27,32,49,56,54,64,12,58,33,4011.0
4365,39,32,30,25,32,49,56,56,64,12,58,33,4011.0
4620,39,31,30,25,33,54,63,60,72,12,62,35,4011.0
4621,39,32,30,25,33,54,63,61,72,12,62,35,4011.0


In [63]:
# Each field has several pixels in| the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id

test_data_grouped = test_data.groupby(['field_id']).mean().reset_index()
test_data_grouped.field_id = [str(int(i)) for i in test_data_grouped.field_id.values]
test_data_grouped

,field_id,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B09,B11,B12
0,11,43.086957,39.057971,38.420290,38.405797,42.507246,59.536232,68.797101,64.405797,76.130435,12.507246,72.159420,48.768116
1,13,44.000000,40.692308,41.307692,46.769231,49.461538,61.076923,70.000000,65.461538,77.923077,13.000000,84.076923,59.307692
2,19,45.469388,41.918367,42.530612,44.367347,47.122449,62.469388,71.979592,66.265306,78.428571,12.000000,78.938776,57.224490
3,21,44.000000,40.250000,40.750000,40.000000,44.250000,64.250000,74.833333,69.583333,81.750000,12.000000,72.166667,46.250000
4,25,47.000000,44.300000,47.200000,52.900000,54.900000,67.300000,77.100000,71.200000,84.800000,13.000000,95.700000,72.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525,7319,45.000000,36.333333,31.222222,24.722222,28.166667,54.888889,71.944444,67.833333,78.555556,14.444444,40.722222,19.944444
1526,7325,45.705882,37.735294,32.205882,26.205882,28.500000,52.823529,69.617647,66.970588,75.823529,13.500000,40.735294,20.235294
1527,7329,47.000000,40.588235,35.823529,30.823529,32.941176,51.705882,64.470588,59.352941,69.176471,11.235294,43.352941,26.352941
1528,7330,46.562500,40.000000,34.750000,29.125000,31.562500,52.250000,67.125000,63.500000,72.875000,11.000000,37.937500,19.125000


# Submit predictions with field_ids and class probabilites

- run predictions with trained model
- pass to multioutput predictions to csv file with field_id as index
- save output file as submission.csv

In [65]:
# # extract crop_id-label dictionary

# with open('ref_agrifieldnet_competition_v1/ref_agrifieldnet_competition_v1_labels_train/ref_agrifieldnet_competition_v1_labels_train_001c1/ref_agrifieldnet_competition_v1_labels_train_001c1.json') as ll:
#     label_json = json.load(ll)

In [66]:
# crop_dict = {asset.get('values')[0]:asset.get('summary') for asset in label_json['assets']['raster_labels']['file:values']}

In [68]:
# crop_dict

In [69]:
crop_dict = {1: 'Wheat',
 2: 'Mustard',
 3: 'Lentil',
 4: 'No Crop',
 6: 'Sugarcane',
 8: 'Garlic',
 15: 'Potato',
 5: 'Green pea',
 16: 'Bersem',
 14: 'Coriander',
 13: 'Gram',
 9: 'Maize',
 36: 'Rice'}

In [70]:
crop_dict

{1: 'Wheat',
 2: 'Mustard',
 3: 'Lentil',
 4: 'No Crop',
 6: 'Sugarcane',
 8: 'Garlic',
 15: 'Potato',
 5: 'Green pea',
 16: 'Bersem',
 14: 'Coriander',
 13: 'Gram',
 9: 'Maize',
 36: 'Rice'}

In [71]:
def labeler(labeled):
    crop_label = np.array([crop_dict.get(f'{int(i)}') for i in labeled])
    return crop_label

In [110]:
predictions = rf.predict_proba(test_data_grouped.drop('field_id', axis=1 ))

crop_columns = [crop_dict.get(i) for i in rf.classes_]

test_df  = pd.DataFrame(columns= ['field_id'] + crop_columns)

test_df['field_id'] = test_data_grouped.field_id

test_df[crop_columns]= predictions 
test_df.to_csv('/content/drive/MyDrive/Agrifield Zindi Challenge/submissionRandomforest.csv', index=False)

In [111]:
test_df.head()

,field_id,Wheat,Mustard,Lentil,No Crop,Green pea,Sugarcane,Garlic,Maize,Gram,Coriander,Potato,Bersem,Rice
0,11,0.251772,0.364978,0.013500,0.255041,0.000417,0.085928,0.002095,0.01439,0.000278,0.00000,0.010769,0.000000,0.000833
1,13,0.544158,0.168497,0.014221,0.227848,0.019992,0.007358,0.001333,0.00000,0.000952,0.00000,0.002429,0.001032,0.012180
2,19,0.276371,0.381016,0.025445,0.195534,0.004623,0.111605,0.002333,0.00000,0.000952,0.00212,0.000000,0.000000,0.000000
3,21,0.054869,0.140603,0.014362,0.656524,0.000000,0.105890,0.002778,0.00000,0.000000,0.00000,0.003624,0.019821,0.001528
4,25,0.269533,0.041396,0.099890,0.582300,0.000000,0.000000,0.000000,0.00000,0.001222,0.00000,0.000000,0.000000,0.005659
